In [40]:
import json
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, select, MetaData, Table
import requests
import sqlalchemy as sa
import urllib
import sql_scripts as ss
from datetime import date, datetime, timedelta
from threading import Thread

In [41]:
def getAuthforWMS(f_data):
    wmsAccess = f_data["wmsAccess"][0]
    return wmsAccess

In [42]:
#gets connections
def getConnforMYSQL(f_data, accessType):
    dialect = pyodbc.drivers()[-1]
    server = f_data[accessType][0]["server"]
    db = f_data[accessType][0]["database"]
    uid = f_data[accessType][0]["uid"]
    pwd = f_data[accessType][0]["pwd"]
    driver = f_data[accessType][0]["dialect_driver"]
    port = f_data[accessType][0]["port"]

    if accessType == "azureAccess":
        connection_string = (
            " Driver={%s}" %dialect +
            "; SERVER=%s" %server + 
            "; Database=%s " %db + 
            "; UID=%s" %uid +
            "; PWD=%s" %pwd
        )
        quoted = urllib.parse.quote_plus(connection_string)
        quoted = f_data[accessType][0]["dialect_driver"] + quoted
        engine = create_engine(quoted, fast_executemany=True).execution_options(isolation_level="AUTOCOMMIT")
    else:
        quoted = driver + uid + ":" + pwd + "@" + server + ":" + str(port) + "/" + db
        engine = create_engine(quoted).execution_options(isolation_level="AUTOCOMMIT")
        
    return engine

In [43]:
#Extracts data from DB
def extractionFunction(conn_integrator, sql_text):
    with conn_integrator.begin():
        str_sql = sa.text(sql_text)
        results = conn_integrator.execute(str_sql)
        columns = results.keys()

        list_columns = []
        list_rows = []
        #print column_names
        for column_name in columns:
            list_columns += [column_name]
        for row in results:
            list_rows += [row]

    return list_rows, list_columns

In [44]:
#get response from API
def setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers):
    #adds default headers
    headers['Accept'] =  "application/json"
    headers['Content-Type'] =  "application/json"   

    completeHTTP = schemeHTTP + baseHTTP + extraHTTP
    response = requests.get(completeHTTP, headers=headers)
    
    return response

In [45]:
class vtexOrder:
    #it's the class that represents a order, with or without PedidoID
    def __init__(self, orderID, pedidoID, jsonResponse, conn_azure):
        self.orderID = orderID
        self.pedidoID = pedidoID
        self.jsonResponse = jsonResponse
        self.conn_azure = conn_azure

        if self.pedidoID is None:
            self.pedidoID = self.getNewPedidoID(conn_azure)
        
    #gets ID_PEDIDO from the SEQ_ID_PEDIDO_VTEX and associates with the orderMainThread
    def getNewPedidoID(self, conn_azure):
        #rows_IdPedidoList, columns_IdPedidoList = extractionFunction(self.conn_azure, ss.queryList[self.file]['sql_querySequence'] )
        #return (rows_IdPedidoList[0])
        return 1

In [46]:
class threadMain:
    #gets a index (a row=dictionary from a list), the columns and rows of the list, a connection to AzureDB, a file name to control which sql_query will be processed
    #and gets a global variables for the output lists - failed and succeded, the output must be global tables because of the multi threading
    #the mode is refering to get or not to get new idPedidos from azureDB
    def __init__(self, row, columns_List, conn_azure, file, outputSuccessTable, outputFailedTable):
        self.file = file
        self.columns_List = columns_List
        self.row = row
        self.conn_azure = conn_azure
        self.outputSuccessTable = outputSuccessTable
        self.outputFailedTable = outputFailedTable
        
        self.t = Thread(target=self.threadMainTask, args=( ))
        self.t.start()
    
    def getThread(self):
        return (self.t)
        
    def threadMainTask(self):
        #redeclare self variables
        columns_List = self.columns_List
        row = self.row
        outputSuccessTable = self.outputSuccessTable
        outputFailedTable = self.outputFailedTable

        #Define key variables for each order        
        for column_index in range(len(columns_List)):
            if columns_List[column_index] == "CONTA":
                account = row[column_index]
            if columns_List[column_index] == "CHAVE":
                key = row[column_index]
            if columns_List[column_index] == "TOKEN":
                token = row[column_index]
            if columns_List[column_index] == "orderId":
                orderID = row[column_index]
            if columns_List[column_index] == "ID_PEDIDO_VTEX":
                pedidoID = row[column_index]

        #Log the mainThread
        print(f'Starting Thread orderID: {orderID}')

        schemeHTTP = "https://"
        baseHTTP = "%s.%s.com.br" % (account, "vtexcommercestable")
        headers = {                
                'X-VTEX-API-AppKey': key,
                'X-VTEX-API-AppToken': token
                }
        extraHTTP = "/api/oms/pvt/orders/%s" %(orderID)

        #get first response
        response = setupAPIrequest(schemeHTTP, baseHTTP, extraHTTP, headers)
        print("response orderID: %s status_code: %s" %(orderID, response.status_code))

        #good Responses
        if response.status_code == 200:
            order = vtexOrder(orderID, pedidoID, response.json(), self.conn_azure)
            globals()[outputSuccessTable] = globals()[outputSuccessTable] + [order]
        #Bad Responses
        else:
            globals()[outputFailedTable].append({'account': account, 'status_code': response.status_code, 'time': datetime.now(), 'order' : orderID })
    

In [47]:
def main (file):
    #open auth file
    auth = open('auth.json')
    auth_load = json.load(auth)

    #getting conn for azure
    engine_azure = getConnforMYSQL(auth_load, "azureAccess")
    conn_azure = engine_azure.connect()

    #-----------------------------------------------------------------------------------------------------------------------------
    #start the List of success and failed orders
    OutputSuccessTable = ss.queryList[file]['OutputSuccess']
    globals()[OutputSuccessTable] = []
    OutputFailedTable = ss.queryList[file]['OutputFailed']
    globals()[OutputFailedTable] = []

    #-----------------------------------------------------------------------------------------------------------------------------
    ##Gets ALL orders from the temp_tables
    rows_List, columns_List = extractionFunction(conn_azure, ss.queryList[file]['sql_query'])

    #list of main threads
    threads = []
    
    for row in rows_List:
        t = threadMain(row, columns_List, file, conn_azure, OutputSuccessTable, OutputFailedTable)
        threads.append(t.getThread())
    
    for t in threads:
        t.join()

    #-----------------------------------------------------------------------------------------------------------------------------
    #After getting the pedidos_vtex and orders from vtex -> check weather the order is new -> place in a temp_table or update existing
    
    #df_orders = pd.DataFrame.from_dict(globals()[ordersOutputSuccessTable])
    #df2_orders = pd.DataFrame.from_dict(globals()[ordersOutputFailedTable])
    
    #fillNan with empty string
    #df_orders = df_orders.fillna("")
    #df2_orders = df2_orders.fillna("")
    for item in globals()[OutputSuccessTable]:
        print (item.pedidoID)

    #df.to_sql(ss.queryList[file]['resultSuccessTable'], engine_azure, if_exists='replace', index=False)
    #if len(globals()["errorList"]) > 0:
    #    df2.to_sql(ss.queryList[file]['resultFailedTable'], engine_azure, if_exists='append', index=False)

    conn_azure.close()
    

In [48]:
if __name__ == "__main__":
    file = "getOrders_temptables"
    main(file)
    print('getOrders_temptables: done')

Starting Thread orderID: 1348160505463-01
Starting Thread orderID: 1348160560234-01
Starting Thread orderID: 1348160652138-01
Starting Thread orderID: 1348160652140-01
Starting Thread orderID: 1348160652142-01
Starting Thread orderID: 1348160652144-01
Starting Thread orderID: 1348160652146-01
Starting Thread orderID: 1348160652148-01
Starting Thread orderID: 1348160652150-01
Starting Thread orderID: 1348170503359-01
Starting Thread orderID: 1348170505465-01
Starting Thread orderID: 1348170560240-01
Starting Thread orderID: 1348170652152-01
Starting Thread orderID: 1348170652158-01
Starting Thread orderID: 1348170652160-01
Starting Thread orderID: 1348170652162-01
Starting Thread orderID: 1348170652164-01
Starting Thread orderID: 1348170652166-01
Starting Thread orderID: 1348170652168-01
Starting Thread orderID: 1348170652170-01
Starting Thread orderID: 1348170652172-01
Starting Thread orderID: 1348170652174-01
Starting Thread orderID: 1348170652176-01
Starting Thread orderID: 134817065